In [1]:
import cv2
import numpy as np
from scipy.spatial import distance as dist

# Define the CentroidTracker class (same as before)
# ... (Please include the CentroidTracker class from the previous code)
class CentroidTracker:
    def __init__(self, max_disappeared=50):
        self.next_object_id = 0
        self.objects = {}
        self.disappeared = {}
        self.max_disappeared = max_disappeared

    def register(self, centroid):
        self.objects[self.next_object_id] = centroid
        self.disappeared[self.next_object_id] = 0
        self.next_object_id += 1

    def deregister(self, object_id):
        del self.objects[object_id]
        del self.disappeared[object_id]

    def update(self, centroids):
        if len(centroids) == 0:
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        if len(self.objects) == 0:
            for centroid in centroids:
                self.register(centroid)
        else:
            object_ids = list(self.objects.keys())
            prev_centroids = list(self.objects.values())
            D = dist.cdist(np.array(prev_centroids), centroids)

            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            used_rows = set()
            used_cols = set()

            for (row, col) in zip(rows, cols):
                if row in used_rows or col in used_cols:
                    continue

                object_id = object_ids[row]
                self.objects[object_id] = centroids[col]
                self.disappeared[object_id] = 0

                used_rows.add(row)
                used_cols.add(col)

            unused_rows = set(range(0, D.shape[0])).difference(used_rows)
            unused_cols = set(range(0, D.shape[1])).difference(used_cols)

            for row in unused_rows:
                object_id = object_ids[row]
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)

            for col in unused_cols:
                self.register(centroids[col])

        return self.objects

# Load YOLOv3 model and configuration (adjust paths accordingly)
net = cv2.dnn.readNet("E:/AI ML internship BEL/yolov3-tiny.weights", "E:/AI ML internship BEL/yolov3-tiny (1).cfg")

# Load COCO class names (or your custom class names)
with open("E:/AI ML internship BEL/coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Create a list to store object centroids and their corresponding classes
object_centroids = []

# Function to calculate the Euclidean distance between two points (same as before)
# ... (Please include the calculate_distance function from the previous code)

# Define the Region of Interest (ROI) coordinates (x, y, width, height) for video
roi_x, roi_y, roi_width, roi_height = 400, 600, 300, 200

# Open a video capture source (0 for webcam, or provide a video file path)
cap = cv2.VideoCapture("E:/AI ML internship BEL/cars.mp4")  # Change to your video source if needed

# Initialize the CentroidTracker
ct = CentroidTracker()

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Crop the frame to the ROI
    roi = frame[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]

    # Get frame dimensions and prepare input blob for YOLO
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Set the input blob for YOLO
    net.setInput(blob)

    # Get layer names
    layer_names = net.getUnconnectedOutLayersNames()

    # Run forward pass to get YOLO detections
    detections = net.forward(layer_names)

    # Reset the list of object centroids for this frame
    object_centroids = []

    # Process YOLO detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.3:
                # Extract bounding box coordinates
                box = obj[0:4] * np.array([width, height, width, height])
                (x, y, w, h) = box.astype("int")

                # Calculate centroid
                centroid_x = x + w / 2
                centroid_y = y + h / 2

                # Check if the centroid is within the ROI
                if roi_x < centroid_x < roi_x + roi_width and roi_y < centroid_y < roi_y + roi_height:
                    # Draw bounding box and label
                    color = (0, 255, 0)  # Green color
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    text = f"{classes[class_id]}: {confidence:.2f}"
                    cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    # Store object centroid and class
                    object_centroids.append((centroid_x, centroid_y, classes[class_id]))

    # Update the CentroidTracker with object centroids
    objects = ct.update(object_centroids)

    # Display the ROI on the frame
    cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 0, 255), 2)

    # Display the result
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ValueError: Unsupported dtype <U32